In [12]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [13]:
# Store csv into a df
city_data_df = pd.read_csv("output_data/cities.csv")
city_data_df = city_data_df[["City","Lat" ,"Lng", "Max Temp", "Humidity", "Cloudiness","Wind Speed"]]
city_data_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,luanda,-8.83,13.24,73.40,83,3,5.82
1,dikson,73.51,80.55,34.72,90,5,2.95
2,hithadhoo,-0.60,73.08,83.32,76,99,7.61
3,codrington,-28.95,153.24,55.99,87,0,9.17
4,tiznit,29.70,-9.73,69.10,92,7,11.77


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [14]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [8]:
# Heatmap of humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=5)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [17]:
# Narrow down cities that fit criteria and drop any results with null values
narrowed_city_df = city_data_df.loc[(city_data_df["Max Temp"] < 80) & (city_data_df["Max Temp"] > 70) \
                                    & (city_data_df["Wind Speed"] < 10) \
                                    & (city_data_df["Cloudiness"] == 0)].dropna()
narrowed_city_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
43,goderich,43.74,-81.71,75.99,54,0,8.99
97,athabasca,54.72,-113.29,79.18,50,0,5.21
180,lusambo,-4.97,23.44,74.68,26,0,2.10
215,praia,-20.25,-43.81,77.00,38,0,5.82
280,bandrele,-12.91,45.19,77.00,69,0,9.17
303,paldiski,59.35,24.08,71.60,77,0,9.17
396,wana,32.31,69.59,71.62,65,0,1.88
427,esmeraldas,-19.76,-44.31,77.00,38,0,5.82
471,zhezkazgan,47.80,67.71,73.06,42,0,8.61
525,sitka,37.17,-99.65,77.00,51,0,1.01


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [19]:
# Create DataFrame called hotel_df to store hotel names along with city, country and coordinates
hotel_df = narrowed_city_df[["City", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df

,City,Lat,Lng,Hotel Name
43,goderich,43.74,-81.71,
97,athabasca,54.72,-113.29,
180,lusambo,-4.97,23.44,
215,praia,-20.25,-43.81,
280,bandrele,-12.91,45.19,
303,paldiski,59.35,24.08,
396,wana,32.31,69.59,
427,esmeraldas,-19.76,-44.31,
471,zhezkazgan,47.80,67.71,
525,sitka,37.17,-99.65,


In [22]:
# Create a params dict that will be updated with each iteration
params = {"key": g_key,
         "type": "lodging",
         "radius": 5000}


# Loop through the cities_pd and run a lat/long search for each city
for index, row in hotel_df.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    latitude = row["Lat"]
    longitude = row["Lng"]

    # update address key value
    params['location'] = f"{latitude},{longitude}"

    # make request
    hotels = requests.get(base_url, params=params).json()
    response = hotels['results']
    
    try:
               
        hotel_df.loc[index, "Hotel Name"] = response[0]['name']
        
    except (KeyError, IndexError):
        
        hotel_df.loc[index, "Hotel Name"] = ""

hotel_df = hotel_df.loc[(hotel_df["Hotel Name"] != ""), :]
hotel_df

,City,Lat,Lng,Hotel Name
43,goderich,43.74,-81.71,Harmony Inn
97,athabasca,54.72,-113.29,Athabasca Lodge Motel
215,praia,-20.25,-43.81,Hotel Pousada de Minas
280,bandrele,-12.91,45.19,Les Baobabs
303,paldiski,59.35,24.08,Pakri Hostel
396,wana,32.31,69.59,Rasa travel wana
427,esmeraldas,-19.76,-44.31,"MinasTec 24 Hrs - Desentupidora em Esmeraldas,..."
471,zhezkazgan,47.80,67.71,Kvartiry Posutochno.


In [25]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]


In [26]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations)

# Add the layer to the map
fig.add_layer(markers)

# Display Map

fig

Figure(layout=FigureLayout(height='420px'))